In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [22]:
dataset=pd.read_csv('train.csv')

In [23]:
dataset.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [24]:
dataset=dataset.dropna()
dataset=dataset.reset_index()

In [25]:
X=dataset.iloc[:,0:4]
y=dataset.iloc[:,-1]

In [26]:
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [27]:
df_copy=X.copy()

In [28]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,18285):
    review=re.sub('[^a-zA-Z]',' ',df_copy['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

In [29]:
corpus[5:7]

['jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time']

In [30]:
from tensorflow.keras.preprocessing.text import one_hot
voc_size=6000
onehot_rep=[one_hot(word,voc_size)for word in corpus]
onehot_rep

[[5320, 2222, 2859, 4582, 4847, 3866, 5566, 5195, 2788, 3049],
 [2251, 2129, 4389, 3911, 4319, 1957, 1492],
 [1976, 5310, 4976, 1189],
 [4613, 2405, 1262, 191, 1271, 312],
 [3189, 4319, 4743, 4203, 5712, 5551, 4319, 1983, 1223, 4679],
 [5550,
  723,
  790,
  2802,
  5585,
  2505,
  5789,
  128,
  3411,
  4589,
  517,
  4988,
  444,
  5913,
  1492],
 [3436, 834, 4773, 5849, 1090, 1371, 3230, 5079, 3806, 1769, 54],
 [4686, 826, 976, 4434, 387, 2549, 2505, 5780, 3806, 1769, 54],
 [3601, 4523, 4558, 915, 5989, 2958, 5135, 2307, 2505, 1539],
 [3, 5710, 3280, 714, 1629, 2096, 2796, 4682],
 [867, 4337, 1746, 4404, 1590, 3933, 5632, 2515, 4287, 4738, 4530],
 [191, 3528, 4847, 2958, 2505, 387],
 [1034, 134, 1654, 4897, 1889, 5938, 5089, 5089, 4639],
 [1650, 5472, 3718, 415, 3522, 2444, 1764, 3806, 1769, 54],
 [1011, 5628, 5743, 4639, 1905, 3806, 1769, 54],
 [4211, 736, 2364, 3804, 2776, 5042, 3681, 1500, 4888, 3753],
 [3489, 3427, 2129],
 [1935, 674, 2180, 5817, 2505, 5821, 5551, 1492],
 [5382,

In [31]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
embed=pad_sequences(onehot_rep,padding='pre',maxlen=20)
print(embed)
print(embed.shape)

[[   0    0    0 ... 5195 2788 3049]
 [   0    0    0 ... 4319 1957 1492]
 [   0    0    0 ... 5310 4976 1189]
 ...
 [   0    0    0 ... 3806 1769   54]
 [   0    0    0 ... 4959 5489  799]
 [   0    0    0 ...  473  862 1981]]
(18285, 20)


In [32]:
embedding_dimensions=40
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(voc_size,embedding_dimensions,input_length=20))
model.add(tf.keras.layers.LSTM(units=100))
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            240000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 296,501
Trainable params: 296,501
Non-trainable params: 0
_________________________________________________________________


In [34]:
X_fin=np.array(embed)
y_fin=np.array(y)

In [35]:
X_fin

array([[   0,    0,    0, ..., 5195, 2788, 3049],
       [   0,    0,    0, ..., 4319, 1957, 1492],
       [   0,    0,    0, ..., 5310, 4976, 1189],
       ...,
       [   0,    0,    0, ..., 3806, 1769,   54],
       [   0,    0,    0, ..., 4959, 5489,  799],
       [   0,    0,    0, ...,  473,  862, 1981]], dtype=int32)

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_fin,y_fin,test_size=0.25,random_state=0)

In [37]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=25,batch_size=32)

Epoch 1/15
429/429 [==============================] - 13s 26ms/step - loss: 0.3866 - accuracy: 0.7956 - val_loss: 0.1800 - val_accuracy: 0.9237
Epoch 2/15
429/429 [==============================] - 11s 25ms/step - loss: 0.1223 - accuracy: 0.9538 - val_loss: 0.1907 - val_accuracy: 0.9210
Epoch 3/15
429/429 [==============================] - 11s 25ms/step - loss: 0.0679 - accuracy: 0.9754 - val_loss: 0.2109 - val_accuracy: 0.9182
Epoch 4/15
429/429 [==============================] - 11s 25ms/step - loss: 0.0412 - accuracy: 0.9859 - val_loss: 0.2918 - val_accuracy: 0.9171
Epoch 5/15
429/429 [==============================] - 11s 25ms/step - loss: 0.0205 - accuracy: 0.9938 - val_loss: 0.3320 - val_accuracy: 0.9167
Epoch 6/15
429/429 [==============================] - 11s 25ms/step - loss: 0.0129 - accuracy: 0.9965 - val_loss: 0.4508 - val_accuracy: 0.9123
Epoch 7/15
429/429 [==============================] - 11s 25ms/step - loss: 0.0098 - accuracy: 0.9970 - val_loss: 0.4746 - val_accuracy:

In [38]:
y_pred=model.predict_classes(X_test)

/home/saransh/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:459: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [39]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[2392,  191],
       [ 248, 1741]])

In [40]:
score=accuracy_score(y_test,y_pred)*100
print(score)

90.39807524059492
